In [1]:
# Dependencies
import gmaps
import pandas as pd
import requests
import json
from pprint import pprint

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key= gkey)

In [2]:
# Import weather data from csv into data frame and dislay data frame
data= pd.read_csv('../Data_Output/city_weather.csv')
data_df=  pd.DataFrame(data)
data_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Fortuna,US,40.5982,-124.1573,50.29,62.0,1.0,12.66
1,Port Hawkesbury,CA,45.6169,-61.3485,34.61,86.0,90.0,26.46
2,Svobodnyy,RU,51.4000,128.1333,35.78,68.0,24.0,15.79
3,Hobart,AU,-42.8794,147.3294,64.69,48.0,0.0,10.36
4,Puerto Ayora,EC,-0.7393,-90.3518,79.00,92.0,75.0,3.00


In [3]:
# Check the data types of the values in the columns
data_df.dtypes

City                 object
Country              object
Latitude            float64
Longitude           float64
Temperature (F)     float64
Humidity (%)        float64
Cloudiness (%)      float64
Wind Speed (mph)    float64
dtype: object

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '375px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [5]:
# Get heat map to display humidity for every city in WeatherPy

locations = data_df[["Latitude", "Longitude"]]
weights = data_df["Humidity (%)"]
fig = gmaps.figure(layout= figure_layout, center=(0,0), zoom_level=2)
heat_layer= gmaps.heatmap_layer(locations, weights= weights, dissipating= False, max_intensity= data_df["Humidity (%)"].max(), point_radius= 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='375px', margin='0 auto 0 auto', padding='1px', wi…

In [6]:
# Narrow data for ideal weather conditons (drop any rows that do not have all conditions):
    # max temperature between 70 and 80
    # wind speed < 10mph
    # zero cloudiness

ideal_weather_df= data_df[(data_df["Temperature (F)"]>=70)  & (data_df["Temperature (F)"]<=80)
                      & (data_df["Wind Speed (mph)"]<=10) & (data_df["Cloudiness (%)"]== 0)
                      ]
# Display data frame
ideal_weather_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
71,Terenos,BR,-20.4422,-54.8603,75.20,73.0,0.0,5.75
154,Bauchi,NG,10.5000,10.0000,70.75,18.0,0.0,6.55
229,Dubbo,AU,-32.2500,148.6167,73.40,56.0,0.0,9.22
246,Jatai,BR,-17.8814,-51.7144,71.28,61.0,0.0,3.71
260,Saint-Philippe,RE,-21.3585,55.7679,70.23,73.0,0.0,5.75


In [7]:
# Make list of dictionaries with the followign informationfor each idea city using .itertuples()
    # City Name, Country, Location (Latitude, Longitude)
ideal_cities_list = [
    {"city": city.City, "country": city.Country, "location": f"{city.Latitude},{city.Longitude}"}
    for city in ideal_weather_df.itertuples()
]
ideal_cities_list

[{'city': 'Terenos', 'country': 'BR', 'location': '-20.4422,-54.8603'},
 {'city': 'Bauchi', 'country': 'NG', 'location': '10.5,10.0'},
 {'city': 'Dubbo', 'country': 'AU', 'location': '-32.25,148.6167'},
 {'city': 'Jatai', 'country': 'BR', 'location': '-17.8814,-51.7144'},
 {'city': 'Saint-Philippe', 'country': 'RE', 'location': '-21.3585,55.7679'},
 {'city': 'Joaíma', 'country': 'BR', 'location': '-16.6542,-41.0306'},
 {'city': 'Caravelas', 'country': 'BR', 'location': '-17.7125,-39.2481'},
 {'city': 'Miandrivazo', 'country': 'MG', 'location': '-19.5167,45.4667'},
 {'city': 'Porecatu', 'country': 'BR', 'location': '-22.7558,-51.3792'},
 {'city': 'Tamworth', 'country': 'AU', 'location': '-31.1,150.9333'},
 {'city': 'Paoua', 'country': 'CF', 'location': '7.2427,16.4406'},
 {'city': 'Saint-Joseph', 'country': 'RE', 'location': '-21.3667,55.6167'},
 {'city': 'Port Lincoln', 'country': 'AU', 'location': '-34.7333,135.8667'},
 {'city': 'Vila Velha', 'country': 'BR', 'location': '-20.3297,-40

In [8]:
# Build queries to send API request
results= []

for entry in ideal_cities_list:
    # get information for the request
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params= {
        "location": entry["location"],
        "radius": "5000",
        "type": "lodging",
        "key": gkey,
    }
    
    # assemble url
    print(f'Retrieving Resulst for {entry["city"]} in {entry["country"]}.')
    response= requests.get(base_url, params= params).json()
    
    # Handle if there is no data for the params passed by adding None to the value of 'lodging'
    if response['results']:
        entry["lodging"]= {"name":response['results'][0]['name'], "location":response['results'][0]['geometry']['location']}
    else:
        entry["lodging"]= None

Retrieving Resulst for Terenos in BR.
Retrieving Resulst for Bauchi in NG.
Retrieving Resulst for Dubbo in AU.
Retrieving Resulst for Jatai in BR.
Retrieving Resulst for Saint-Philippe in RE.
Retrieving Resulst for Joaíma in BR.
Retrieving Resulst for Caravelas in BR.
Retrieving Resulst for Miandrivazo in MG.
Retrieving Resulst for Porecatu in BR.
Retrieving Resulst for Tamworth in AU.
Retrieving Resulst for Paoua in CF.
Retrieving Resulst for Saint-Joseph in RE.
Retrieving Resulst for Port Lincoln in AU.
Retrieving Resulst for Vila Velha in BR.
Retrieving Resulst for Itaunja in IN.
Retrieving Resulst for Abu Dhabi in AE.
Retrieving Resulst for Guaratinga in BR.
Retrieving Resulst for Itapirapuã in BR.
Retrieving Resulst for Mineiros in BR.
Retrieving Resulst for Saint-Louis in RE.
Retrieving Resulst for Ruwi in OM.
Retrieving Resulst for Inverell in AU.
Retrieving Resulst for Saint-Pierre in RE.
Retrieving Resulst for Coxim in BR.


In [9]:
# Display list of dictionaries
ideal_cities_list

[{'city': 'Terenos',
  'country': 'BR',
  'location': '-20.4422,-54.8603',
  'lodging': {'name': 'QUINTAS',
   'location': {'lat': -20.4383001, 'lng': -54.8652419}}},
 {'city': 'Bauchi', 'country': 'NG', 'location': '10.5,10.0', 'lodging': None},
 {'city': 'Dubbo',
  'country': 'AU',
  'location': '-32.25,148.6167',
  'lodging': {'name': 'Comfort Inn Blue Lagoon',
   'location': {'lat': -32.25444839999999, 'lng': 148.6070196}}},
 {'city': 'Jatai',
  'country': 'BR',
  'location': '-17.8814,-51.7144',
  'lodging': {'name': 'La Vitre Hotel',
   'location': {'lat': -17.8871304, 'lng': -51.7122693}}},
 {'city': 'Saint-Philippe',
  'country': 'RE',
  'location': '-21.3585,55.7679',
  'lodging': {'name': 'Chambres d\'hôte "La Trinité"',
   'location': {'lat': -21.358434, 'lng': 55.77032209999999}}},
 {'city': 'Joaíma',
  'country': 'BR',
  'location': '-16.6542,-41.0306',
  'lodging': {'name': 'Pousada LAZO',
   'location': {'lat': -16.6567972, 'lng': -41.02939359999999}}},
 {'city': 'Carave

In [10]:
# Plot the lodging on top of the humidity heatmap with each pin containing the Lodging Name, City, and Country.

# locations of ideal cities from data_df data frame and weights = humidity
locations = data_df[["Latitude", "Longitude"]]
weights = data_df["Humidity (%)"]
# Build the heat_layer
heat_layer= gmaps.heatmap_layer(locations, weights= weights, dissipating= False, max_intensity= data_df["Humidity (%)"].max(), point_radius= 1)
fig.add_layer(heat_layer)

# Build the lodging locations
ideal_lodging_locations= [(city['lodging']['location']['lat'], city['lodging']['location']['lng']) for city in ideal_cities_list if city['lodging']]

#Build the info box text
info_box_template= """
<dl>
<dt>Lodging Name</d><dd>{name}</dd><br>
<dt>City</d><dd>{city}</dd>
<dt>Country</d><dd>{country}</dd>
</dl>
"""
lodging_info= [info_box_template.format(**city, name= city['lodging']['name']) for city in ideal_cities_list if city['lodging']]

# Build the marker layer
marker_layer= gmaps.marker_layer(ideal_lodging_locations, info_box_content= lodging_info)

# Make the figure and add layers
fig= gmaps.figure(layout= figure_layout, center=(0,0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figures
fig

Figure(layout=FigureLayout(border='1px solid black', height='375px', margin='0 auto 0 auto', padding='1px', wi…